In [1]:
from services import service_api

In [2]:
service_api.get_shopping_list(['poulet curry','quiche aux lardons'],4)

{'name': 'Filet de poulet au curry', 'url': 'http://www.marmiton.org/recettes/recette_filet-de-poulet-au-curry_80990.aspx', 'ingredients': [{'name': 'poivre', 'quantity': '', 'unit': ''}, {'name': 'sel', 'quantity': '', 'unit': ''}, {'name': 'curry', 'quantity': '1', 'unit': 'cuillère à café'}, {'name': "huile d'olive", 'quantity': '1', 'unit': 'cuillère à soupe'}, {'name': 'oignon', 'quantity': '1', 'unit': ''}, {'name': 'filets de poulet', 'quantity': '4', 'unit': ''}, {'name': 'ail', 'quantity': '', 'unit': ''}, {'name': 'sauce soja', 'quantity': '4', 'unit': 'cuillères à soupe'}, {'name': 'ciboulette', 'quantity': '4', 'unit': 'brins'}, {'name': 'crème fraîche épaisse', 'quantity': '10', 'unit': 'cl'}], 'nb_persons': '4'}
{'name': 'Quiche poireaux, chèvre lardons', 'url': 'http://www.marmiton.org/recettes/recette_quiche-poireaux-chevre-lardons_22275.aspx', 'ingredients': [{'name': 'sel', 'quantity': '', 'unit': ''}, {'name': 'poivre', 'quantity': '', 'unit': ''}, {'name': 'crème fr

[{'name': 'poivre', 'quantity': '', 'unit': ''},
 {'name': 'sel', 'quantity': '', 'unit': ''},
 {'name': 'curry', 'quantity': '1', 'unit': 'cuillère à café'},
 {'name': "huile d'olive", 'quantity': '1', 'unit': 'cuillère à soupe'},
 {'name': 'oignon', 'quantity': '1', 'unit': ''},
 {'name': 'filets de poulet', 'quantity': '4', 'unit': ''},
 {'name': 'ail', 'quantity': '', 'unit': ''},
 {'name': 'sauce soja', 'quantity': '4', 'unit': 'cuillères à soupe'},
 {'name': 'ciboulette', 'quantity': '4', 'unit': 'brins'},
 {'name': 'crème fraîche épaisse', 'quantity': '10', 'unit': 'cl'},
 {'name': 'crème fraîche', 'quantity': '20', 'unit': 'cl'},
 {'name': 'poireaux', 'quantity': '3', 'unit': ''},
 {'name': 'pâte brisée', 'quantity': '250', 'unit': 'g'},
 {'name': 'lardons fumés', 'quantity': '100', 'unit': 'g'},
 {'name': 'fromage de chèvre', 'quantity': '1', 'unit': ''},
 {'name': 'oeufs', 'quantity': '3', 'unit': ''},
 {'name': 'lait', 'quantity': '10', 'unit': 'cl'}]

In [3]:
service_api.get_shopping_list(['poulet curry','quiche aux lardons','quiche aux légumes de soleil'],6)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


{'name': 'Filet de poulet au curry', 'url': 'http://www.marmiton.org/recettes/recette_filet-de-poulet-au-curry_80990.aspx', 'ingredients': [{'name': 'poivre', 'quantity': '', 'unit': ''}, {'name': 'sel', 'quantity': '', 'unit': ''}, {'name': 'curry', 'quantity': '1', 'unit': 'cuillère à café'}, {'name': "huile d'olive", 'quantity': '1', 'unit': 'cuillère à soupe'}, {'name': 'oignon', 'quantity': '1', 'unit': ''}, {'name': 'filets de poulet', 'quantity': '4', 'unit': ''}, {'name': 'ail', 'quantity': '', 'unit': ''}, {'name': 'sauce soja', 'quantity': '4', 'unit': 'cuillères à soupe'}, {'name': 'ciboulette', 'quantity': '4', 'unit': 'brins'}, {'name': 'crème fraîche épaisse', 'quantity': '10', 'unit': 'cl'}], 'nb_persons': '4'}
{'name': 'Quiche poireaux, chèvre lardons', 'url': 'http://www.marmiton.org/recettes/recette_quiche-poireaux-chevre-lardons_22275.aspx', 'ingredients': [{'name': 'sel', 'quantity': '', 'unit': ''}, {'name': 'poivre', 'quantity': '', 'unit': ''}, {'name': 'crème fr

[{'name': 'poivre', 'quantity': '', 'unit': ''},
 {'name': 'sel', 'quantity': '', 'unit': ''},
 {'name': 'curry', 'quantity': 1.5, 'unit': 'cuillère à café'},
 {'name': "huile d'olive", 'quantity': 1.5, 'unit': 'cuillère à soupe'},
 {'name': 'oignon', 'quantity': 3.0, 'unit': ''},
 {'name': 'filets de poulet', 'quantity': 6.0, 'unit': ''},
 {'name': 'ail', 'quantity': '', 'unit': ''},
 {'name': 'sauce soja', 'quantity': 6.0, 'unit': 'cuillères à soupe'},
 {'name': 'ciboulette', 'quantity': 6.0, 'unit': 'brins'},
 {'name': 'crème fraîche épaisse', 'quantity': 15.0, 'unit': 'cl'},
 {'name': 'crème fraîche', 'quantity': 30.0, 'unit': 'cl'},
 {'name': 'poireaux', 'quantity': 4.5, 'unit': ''},
 {'name': 'pâte brisée', 'quantity': 375.0, 'unit': 'g'},
 {'name': 'lardons fumés', 'quantity': 150.0, 'unit': 'g'},
 {'name': 'fromage de chèvre', 'quantity': 1.5, 'unit': ''},
 {'name': 'oeufs', 'quantity': 7.5, 'unit': ''},
 {'name': 'lait', 'quantity': 15.0, 'unit': 'cl'},
 {'name': "huile d'oliv